# Step 1: Import relevant pytorch libraries for the process


In [98]:
import torch
import torchvision
import torch.utils.data
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

# Step 2: Gather the relevant dataset from torch vision in order to train and test the built model

In [99]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.48216, 0.44653)
                                                                           ,(0.24703, 0.24349, 0.26159))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=100,shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,shuffle=False, num_workers=0)






Files already downloaded and verified
Files already downloaded and verified


# Step 3: Build the model with 3 layers including 1 hidden

In [100]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(32*32*3, 500) #the size of a single data entry as input layer units
        self.fc2 = nn.Linear(500, 10) #a hidden layer and a 10 class output
        

    def forward(self, x):
        x = F.relu(self.fc1(x)) #activation function ReLU used to cater non-linearity
        x = self.fc2(x)
        return x

# Step 4: Train the model using the CIFAR-10 dataset as downloaded

In [101]:
model = Net()   #create an object of class Net
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4) #gradient descent optimizing function with tuned learning rate
criterion = nn.CrossEntropyLoss() #classification loss function

for iterations in range(10): 
    for batch_id, data_target in enumerate(trainloader):
      inputdata = data_target[0] #image
      target = data_target[1] #label of the data entry in the batch
      inputdata = inputdata.view(-1, 32 * 32 * 3) #convert incoming images to vectors
      optimizer.zero_grad() # so gradients from previous iterations are equated to zero for new iteration

      # Complete a forward pass
      output = model(inputdata)

      # Compute the loss, gradients and change the weights
      loss = criterion(output,target)
      loss.backward() #compute new gradients
      optimizer.step() #adjust the weights
       

# Step 5: Test/Evaluate the model with the loaded test dataset from CIFAR-10

In [102]:
# Set the model in evaluation mode
model.eval() 
correct, total = 0, 0
predictions = []

for i, data in enumerate(testloader, 0):
      inputs, labels = data
    
      # Put each image into a vector
      inputs = inputs.view(-1, 32*32*3)
    
      # Do the forward pass and get the predictions
      outputs = model(inputs)
      _, outputs = torch.max(outputs.data,1)
      total += labels.size(0)
      correct += (outputs == labels).sum().item()
print('The testing set accuracy of the network is: %d %%' % (100 * correct / total))

The testing set accuracy of the network is: 52 %
